# eCommerce Product Recommendation - Part 2

# 1. Load Raw Data 

In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import OrderedDict

In [ ]:
order_products_prior = pd.read_csv('order_products_prior.csv') 
order_products_train = pd.read_csv('order_products_train.csv') 
orders = pd.read_csv('orders.csv') 
products = pd.read_csv('products.csv') 

In [ ]:
order_products_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,12,30597,1,1
1,12,15221,2,1
2,12,43772,3,1
3,12,37886,4,1
4,12,37215,5,0


In [ ]:
order_products_train.head()

,order_id,product_id,add_to_cart_order,reordered
0,1077,13176,1,1
1,1077,39922,2,1
2,1077,5258,3,1
3,1077,21137,4,1
4,1119,6046,1,1


# 1. Construct Model Label

In [ ]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,1363380,50,prior,1,3,9,NaN
1,3131103,50,prior,2,6,12,10.0
2,2197066,50,prior,3,1,13,9.0
3,3201640,50,prior,4,0,11,6.0
4,2756806,50,prior,5,4,14,11.0


In [ ]:
order_products_train.head()

,order_id,product_id,add_to_cart_order,reordered
0,1077,13176,1,1
1,1077,39922,2,1
2,1077,5258,3,1
3,1077,21137,4,1
4,1119,6046,1,1


In [ ]:
train_details = order_products_train.merge(orders, on = 'order_id')  
prior_details = order_products_prior.merge(orders, on = 'order_id') 

In [ ]:
train_user_ids = set(orders[orders['eval_set'] == 'train']['user_id']) 
model_all_data = prior_details[prior_details.user_id.isin(train_user_ids)][['user_id','product_id','order_number', 'order_dow', 'order_hour_of_day', 'days_since_prior_order']]
model_all_data = model_all_data.drop_duplicates(subset=['user_id', 'product_id']) 

Build unique keys

In [ ]:
model_all_data['unique_key'] = model_all_data['user_id'].astype('str') + '_' + model_all_data['product_id'].astype('str') 
train_unique_key = train_details['user_id'].astype('str') + '_' + train_details['product_id'].astype('str') 

Labelling

In [ ]:
model_all_data['label'] = 0 
model_all_data.loc[model_all_data.unique_key.isin(train_unique_key), 'label'] = 1 

In [ ]:
model_all_data.head()

,user_id,product_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,unique_key,label
0,152610,30597,22,6,8,10.0,152610_30597,0
1,152610,15221,22,6,8,10.0,152610_15221,0
2,152610,43772,22,6,8,10.0,152610_43772,0
3,152610,37886,22,6,8,10.0,152610_37886,0
4,152610,37215,22,6,8,10.0,152610_37215,0


# 2. Construct Model Features 

2.1. Feature Group 1: user-product activity features 

In [ ]:
user_product_features = ['user_product__total_orders',
                         'user_product__add_to_cart_order_mean',
                         'user_product__reordered_mean',
                         'user_product__most_dow',
                         'user_product__most_hod']

In [ ]:
prior_details.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,12,30597,1,1,152610,prior,22,6,8,10.0
1,12,15221,2,1,152610,prior,22,6,8,10.0
2,12,43772,3,1,152610,prior,22,6,8,10.0
3,12,37886,4,1,152610,prior,22,6,8,10.0
4,12,37215,5,0,152610,prior,22,6,8,10.0


In [ ]:
df_user_product_features = (prior_details.groupby(['product_id','user_id'],as_index=False) 
                                           .agg(OrderedDict(
                                                   [('order_id','count'), 
                                                    ('add_to_cart_order','mean'), 
                                                    ('reordered', 'mean'), 
                                                    ('order_dow', (lambda x: x.mode()[0])), 
                                                    ('order_hour_of_day', (lambda x: x.mode()[0])), 
                                                    ])))
df_user_product_features.columns = ['product_id', 'user_id'] + user_product_features

In [ ]:
model_all_data = model_all_data.merge(df_user_product_features, on = ['user_id', 'product_id'])

2.2. Feature Group 2: product features 

In [ ]:
product_features = ['product__total_orders',
                     'product__add_to_cart_order_mean',
                     'product__total_users',
                     'product__reordered_mean',
                     'product__most_dow',
                     'product__most_hod',
                     'product__days_since_prior_order_mean'
                     ]

In [ ]:
df_product_features = (prior_details.groupby(['product_id'],as_index=False)
                                           .agg(OrderedDict(
                                                   [('order_id','nunique'),
                                                    ('add_to_cart_order','mean'), 
                                                    ('user_id', 'nunique'),
                                                    ('reordered', 'mean'), 
                                                    ('order_dow', (lambda x: x.mode()[0])), 
                                                    ('order_hour_of_day', (lambda x: x.mode()[0])), 
                                                    ('days_since_prior_order', 'mean') 
                                                    ])))
df_product_features.columns = ['product_id'] + product_features

In [ ]:
model_all_data = model_all_data.merge(df_product_features, on = ['product_id'])

In [ ]:
model_all_data = model_all_data.merge(products[['product_id','aisle_id', 'department_id']], 
                                      on = ['product_id'])

In [ ]:
model_all_data.rename(columns={'aisle_id': 'product__aisle_id', 'department_id': 'product__department_id'}, inplace=True)

2.3. Feature Group 3: user features

In [ ]:
user_features = ['user__order_count',
                  'user__product_count',
                  'user__days_since_prior_order_mean',
                  'user__reordered_mean',
                  'user__most_dow',
                  'user__most_hod',
                  ]

In [ ]:
df_user_features = (prior_details.groupby(['user_id'],as_index=False)
                                           .agg(OrderedDict(
                                                   [('order_id','nunique'), 
                                                    ('product_id','count'), 
                                                    ('days_since_prior_order','mean'), 
                                                    ('reordered', 'mean'),
                                                    ('order_dow', (lambda x: x.mode()[0])), 
                                                    ('order_hour_of_day', (lambda x: x.mode()[0])), 
                                                    ])))
df_user_features.columns = ['user_id'] + user_features

In [ ]:
model_all_data = model_all_data.merge(df_user_features, on = ['user_id'])

In [ ]:
model_all_data['label'].value_counts()

0    297467
1     32339
Name: label, dtype: int64

In [ ]:
model_all_data.shape

(329806, 28)

In [ ]:
model_all_data.head()

,user_id,product_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,unique_key,label,user_product__total_orders,user_product__add_to_cart_order_mean,...,product__most_hod,product__days_since_prior_order_mean,product__aisle_id,product__department_id,user__order_count,user__product_count,user__days_since_prior_order_mean,user__reordered_mean,user__most_dow,user__most_hod
0,152610,30597,22,6,8,10.0,152610_30597,0,5,2.00,...,9,12.592593,53,16,26,408,10.953804,0.375,6,12
1,152610,15221,22,6,8,10.0,152610_15221,0,8,8.25,...,12,13.653846,84,16,26,408,10.953804,0.375,6,12
2,152610,43772,22,6,8,10.0,152610_43772,0,2,3.00,...,12,12.008565,83,4,26,408,10.953804,0.375,6,12
3,152610,37886,22,6,8,10.0,152610_37886,0,2,3.50,...,14,11.489362,53,16,26,408,10.953804,0.375,6,12
4,152610,37215,22,6,8,10.0,152610_37215,0,1,5.00,...,13,11.490066,98,7,26,408,10.953804,0.375,6,12
